In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers biopython kaggle
print("✅ Đã cài đặt xong thư viện!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 44.7 MB/s eta 0:00:00
✅ Đã cài đặt xong thư viện!


In [2]:
import os
from google.colab import files

# Upload file kaggle.json
print("Vui lòng upload file kaggle.json của bạn:")
files.upload()

# Cấu hình Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Tải dữ liệu cuộc thi (Sẽ mất khoảng 1-2 phút)
print("⏳ Đang tải dữ liệu CAFA 6...")
!kaggle competitions download -c cafa-6-protein-function-prediction
!unzip -q cafa-6-protein-function-prediction.zip -d /content/cafa6_data
print("✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data")

Vui lòng upload file kaggle.json của bạn:


Saving kaggle.json to kaggle.json
⏳ Đang tải dữ liệu CAFA 6...
 73% 67.0M/91.2M [00:00<00:00, 683MB/s]
100% 91.2M/91.2M [00:00<00:00, 627MB/s]
✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data


In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm.auto import tqdm
import gc
import os
import sys

# ============================================================================
# CONFIG CHO ANKH-BASE
# ============================================================================
CONFIG = {
    # Đường dẫn dữ liệu (Nơi bạn đã gộp file FINAL)
    'EMBED_DIR': '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged',
    'SAVE_DIR': '/content/drive/MyDrive/CAFA6_Results/Ankh_Base_Run',
    'TRAIN_TERMS': '/content/cafa6_data/Train/train_terms.tsv',

    # Model Params
    'TOP_K_LABELS': 10000,

    # Kiến trúc mạng: Điều chỉnh cho Ankh-Base (Input 768)
    # Giảm xuống [1024, 512, 256] là đủ mạnh và train nhanh
    'HIDDEN_DIMS': [1024, 512, 256],

    'DROPOUT_RATE': 0.25,         # Dropout chuẩn
    'EPOCHS': 40,
    'BATCH_SIZE': 128,            # Tăng batch size lên được nếu RAM dư
    'LEARNING_RATE': 1e-3,        # Tăng LR lên lại 1e-3 vì mạng nhỏ hơn dễ học hơn
    'LABEL_SMOOTHING': 0.1,

    # Inference Params
    'TEMPERATURE': 1.0,
    'MIN_CONFIDENCE': 0.01,
    'MAX_PREDS_PER_PROTEIN': 20,

    'SEED': 42,
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu'
}

# Tạo thư mục lưu kết quả
os.makedirs(CONFIG['SAVE_DIR'], exist_ok=True)

print("="*80)
print(f"🚀 CAFA 6 - ANKH-BASE TRAINING | Device: {CONFIG['DEVICE']}")
print("="*80)

# Set seed
torch.manual_seed(CONFIG['SEED'])
np.random.seed(CONFIG['SEED'])

# ============================================================================
# 1. LOAD DATA (ANKH BASE)
# ============================================================================
print("\n[1/5] Loading Ankh Data (Optimized)...")

try:
    # 1. Dùng mmap_mode='r' để KHÔNG load toàn bộ file vào RAM ngay lập tức
    # Chỉ khi nào cần dùng mới đọc từ đĩa
    print("   ⏳ Mapping Embeddings from disk...")
    X_train_mmap = np.load(f"{CONFIG['EMBED_DIR']}/ankh_train_FINAL_embeddings.npy", mmap_mode='r')
    train_ids = np.load(f"{CONFIG['EMBED_DIR']}/ankh_train_FINAL_ids.npy")

    # Test set không cần load bây giờ, để dành lúc inference
    # X_test_embeds = ... (Bỏ qua để tiết kiệm RAM)
    # test_ids = ...

    print(f"   ✓ Train Embeds Mapped: {X_train_mmap.shape}")

except FileNotFoundError:
    print("❌ LỖI: Không tìm thấy file FINAL!")
    sys.exit(1)

# ============================================================================
# 2. PROCESS LABELS (IA STRATEGY)
# ============================================================================
print("\nProcessing Labels with IA Strategy...")

df_terms = pd.read_csv(CONFIG['TRAIN_TERMS'], sep='\t', header=0, names=['EntryID', 'term', 'aspect'])
df_ia = pd.read_csv('/content/cafa6_data/IA.tsv', sep='\t', names=['term', 'ia'])
ia_dict = dict(zip(df_ia['term'], df_ia['ia']))

# Tính Score = Freq * IA
term_counts = df_terms['term'].value_counts().reset_index()
term_counts.columns = ['term', 'freq']
term_counts['ia'] = term_counts['term'].map(ia_dict).fillna(0.0)
term_counts['score'] = term_counts['freq'] * term_counts['ia']

# Chọn Top K
top_terms_df = term_counts.sort_values(by='score', ascending=False).head(CONFIG['TOP_K_LABELS'])
top_terms = top_terms_df['term'].tolist()

print(f"   ✓ Selected {len(top_terms)} terms.")

# Lọc và tạo dict
df_filtered = df_terms[df_terms['term'].isin(top_terms)]
id_to_terms = df_filtered.groupby('EntryID')['term'].apply(list).to_dict()

# Dọn dẹp biến trung gian ngay lập tức
del df_terms, df_ia, term_counts, top_terms_df, df_filtered
gc.collect()

# ============================================================================
# 3. PREPARE TENSORS (SPARSE & FLOAT32)
# ============================================================================
print("\n[3/5] Preparing Tensors (Memory Safe)...")

# 1. Dùng Sparse Output để tiết kiệm RAM lúc tạo ma trận
mlb = MultiLabelBinarizer(classes=top_terms, sparse_output=True)
mlb.fit([top_terms])

# 2. Transform (Ra ma trận thưa - tốn rất ít RAM)
print("   ⏳ Transforming to Sparse Matrix...")
y_labels_list = [id_to_terms.get(pid, []) for pid in train_ids]
y_train_sparse = mlb.transform(y_labels_list)

# Xóa list python nặng nề
del y_labels_list
gc.collect()

# 3. Tạo Weights
weights_list = [ia_dict.get(t, 0.0) for t in mlb.classes_]
pos_weight_tensor = torch.tensor(weights_list, dtype=torch.float32).to(CONFIG['DEVICE'])

# 4. Convert sang Dense Float32 (Tiết kiệm 50% RAM so với float64 mặc định)
print("   ⏳ Converting to Dense Float32...")
y_train_binary = y_train_sparse.astype(np.float32).toarray()
del y_train_sparse
gc.collect()

# 5. Tạo Tensor (Dùng from_numpy để không copy dữ liệu)
y_train_tensor = torch.from_numpy(y_train_binary)

# Label Smoothing (Thực hiện in-place trên Tensor để không tốn RAM copy)
if CONFIG['LABEL_SMOOTHING'] > 0:
    print("   ⏳ Applying Label Smoothing in-place...")
    y_train_tensor.mul_(1 - CONFIG['LABEL_SMOOTHING']).add_(CONFIG['LABEL_SMOOTHING'] / len(top_terms))

# Xóa bản numpy gốc vì giờ đã có tensor (giữ tensor thôi)
del y_train_binary
gc.collect()

# 6. Xử lý Input X (Load từ mmap vào RAM dưới dạng float32)
print("   ⏳ Loading X into RAM as float32...")
# Copy từ đĩa vào RAM và ép kiểu cùng lúc
X_train_tensor = torch.tensor(X_train_mmap, dtype=torch.float32)
# Xóa biến mmap
del X_train_mmap
gc.collect()

# ============================================================================
# 4. DATA SPLIT (INDICES ONLY)
# ============================================================================
# Thay vì train_test_split dữ liệu (nhân đôi RAM), ta chỉ split Index
print("   ⏳ Splitting Indices...")
indices = np.arange(len(X_train_tensor))
train_idx, val_idx = train_test_split(indices, test_size=0.15, random_state=CONFIG['SEED'])

# Tạo Dataset từ Subset (Không copy dữ liệu)
train_dataset = TensorDataset(X_train_tensor[train_idx], y_train_tensor[train_idx])
val_dataset = TensorDataset(X_train_tensor[val_idx], y_train_tensor[val_idx])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=False)

print(f"   ✓ Ready: Train={len(train_idx)}, Val={len(val_idx)}")

# ============================================================================
# 2. MODEL ARCHITECTURE
# ============================================================================
print("\n[2/5] Building MLP Model...")

class AnkhMLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims, dropout):
        super().__init__()
        layers = []
        prev_dim = input_dim

        for dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, dim),
                nn.BatchNorm1d(dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = dim

        self.encoder = nn.Sequential(*layers)
        self.output = nn.Linear(prev_dim, output_dim)

    def forward(self, x):
        return self.output(self.encoder(x))

model = AnkhMLP(
    input_dim=X_train_tensor.shape[1],
    output_dim=CONFIG['TOP_K_LABELS'],
    hidden_dims=CONFIG['HIDDEN_DIMS'],
    dropout=CONFIG['DROPOUT_RATE']
).to(CONFIG['DEVICE'])

# ============================================================================
# 3. TRAINING
# ============================================================================
print("\n" + "="*80)
print(f"TRAINING START ({CONFIG['EPOCHS']} Epochs)")
print("="*80)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
optimizer = optim.AdamW(model.parameters(), lr=CONFIG['LEARNING_RATE'], weight_decay=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

best_val_loss = float('inf')
best_model_path = f"{CONFIG['SAVE_DIR']}/ankh_base_best.pth"

for epoch in range(CONFIG['EPOCHS']):
    # Train
    model.train()
    train_loss = 0
    for X_b, y_b in train_loader:
        X_b, y_b = X_b.to(CONFIG['DEVICE']), y_b.to(CONFIG['DEVICE'])

        optimizer.zero_grad()
        logits = model(X_b)
        loss = criterion(logits, y_b)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        train_loss += loss.item()

    avg_train = train_loss / len(train_loader)

    # Val
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_b, y_b in val_loader:
            X_b, y_b = X_b.to(CONFIG['DEVICE']), y_b.to(CONFIG['DEVICE'])
            logits = model(X_b)
            loss = criterion(logits, y_b)
            val_loss += loss.item()

    avg_val = val_loss / len(val_loader)

    scheduler.step(avg_val)
    curr_lr = optimizer.param_groups[0]['lr']

    print(f"Epoch {epoch+1:02d} | Train: {avg_train:.4f} | Val: {avg_val:.4f} | LR: {curr_lr:.1e}")

    if avg_val < best_val_loss:
        best_val_loss = avg_val
        torch.save(model.state_dict(), best_model_path)
        print("  ⭐ New Best Model!")

print(f"\n✅ Training Done! Best Loss: {best_val_loss:.4f}")
del optimizer, scheduler, criterion, train_loader, val_loader
gc.collect()
torch.cuda.empty_cache()

🚀 CAFA 6 - ANKH-BASE TRAINING | Device: cuda

[1/5] Loading Ankh Data (Optimized)...
   ⏳ Mapping Embeddings from disk...
   ✓ Train Embeds Mapped: (82404, 768)

Processing Labels with IA Strategy...
   ✓ Selected 10000 terms.

[3/5] Preparing Tensors (Memory Safe)...
   ⏳ Transforming to Sparse Matrix...
   ⏳ Converting to Dense Float32...
   ⏳ Applying Label Smoothing in-place...
   ⏳ Loading X into RAM as float32...
   ⏳ Splitting Indices...
   ✓ Ready: Train=70043, Val=12361

[2/5] Building MLP Model...

TRAINING START (40 Epochs)
Epoch 01 | Train: 0.0248 | Val: 0.0057 | LR: 1.0e-03
  ⭐ New Best Model!
Epoch 02 | Train: 0.0057 | Val: 0.0054 | LR: 1.0e-03
  ⭐ New Best Model!
Epoch 03 | Train: 0.0055 | Val: 0.0052 | LR: 1.0e-03
  ⭐ New Best Model!
Epoch 04 | Train: 0.0052 | Val: 0.0050 | LR: 1.0e-03
  ⭐ New Best Model!
Epoch 05 | Train: 0.0050 | Val: 0.0048 | LR: 1.0e-03
  ⭐ New Best Model!
Epoch 06 | Train: 0.0048 | Val: 0.0047 | LR: 1.0e-03
  ⭐ New Best Model!
Epoch 07 | Train: 0.0

In [2]:
X_test_embeds = np.load(f"{CONFIG['EMBED_DIR']}/ankh_test_FINAL_embeddings.npy")

test_ids = np.load(f"{CONFIG['EMBED_DIR']}/ankh_test_FINAL_ids.npy")

# ============================================================================
# 4. PREDICTION & SUBMISSION
# ============================================================================
print("\n[4/5] Predicting & Creating Submission...")

model.load_state_dict(torch.load(best_model_path))
model.eval()

# Dataset cho Test
test_dataset = TensorDataset(torch.FloatTensor(X_test_embeds))
test_loader = DataLoader(test_dataset, batch_size=CONFIG['BATCH_SIZE']*2, shuffle=False, num_workers=2)

submission_path = f"{CONFIG['SAVE_DIR']}/submission.tsv"
n_predictions = 0

with open(submission_path, 'w') as f:
    current_idx = 0
    with torch.no_grad():
        for (X_b,) in tqdm(test_loader, desc="Inference"):
            X_b = X_b.to(CONFIG['DEVICE'])
            logits = model(X_b)
            probs_batch = torch.sigmoid(logits / CONFIG['TEMPERATURE']).cpu().numpy()

            # Mapping IDs
            batch_len = len(probs_batch)
            ids_batch = test_ids[current_idx : current_idx + batch_len]
            current_idx += batch_len

            for i, pid in enumerate(ids_batch):
                probs = probs_batch[i]

                # Logic chọn Top K
                top_k = min(len(probs), CONFIG['MAX_PREDS_PER_PROTEIN'])
                ind = np.argpartition(probs, -top_k)[-top_k:]
                ind = ind[np.argsort(probs[ind])][::-1]

                for idx in ind:
                    score = probs[idx]
                    if score > CONFIG['MIN_CONFIDENCE']:
                        f.write(f"{pid}\t{top_terms[idx]}\t{score:.3f}\n")
                        n_predictions += 1

            del probs_batch, X_b, logits

print(f"\n✅ DONE! Predictions: {n_predictions:,}")
print(f"👉 Submission: {submission_path}")


[4/5] Predicting & Creating Submission...


Inference:   0%|          | 0/877 [00:00<?, ?it/s]


✅ DONE! Predictions: 4,421,549
👉 Submission: /content/drive/MyDrive/CAFA6_Results/Ankh_Base_Run/submission.tsv


In [3]:
# Chạy lệnh này trong một cell mới
!kaggle competitions submit \
    -c cafa-6-protein-function-prediction \
    -f /content/drive/MyDrive/CAFA6_Results/Ankh_Base_Run/submission.tsv \
    -m "ankh top-k=20 top-k-label=10k"

100% 102M/102M [00:02<00:00, 50.5MB/s] 
Successfully submitted to CAFA 6 Protein Function Prediction